In [12]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [13]:
load_dotenv()

True

In [14]:
model = ChatOpenAI()

In [15]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [16]:
def create_outline(state : BlogState) -> BlogState:

    # fetch title
    title = state['title']

    prompt = f'Generate an outline for a blog on the topic {title}'

    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [17]:
def create_blog(state: BlogState) -> BlogState:
    title = state["title"]
    outline = state["outline"]

    prompt = f"write a detailed blog on the title - {title} using the following outline \n {outline}"
    content = model.invoke(prompt).content

    state["content"] = content

    return state

In [18]:
graph = StateGraph(BlogState)

# add nodes
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)


# create edges
graph.add_edge(start_key=START, end_key="create_outline")
graph.add_edge(start_key="create_outline", end_key="create_blog")
graph.add_edge(start_key="create_blog", end_key=END)


workflow = graph.compile()

In [19]:
initial_state = {'title' : 'Obesity'}

result  = workflow.invoke(input=initial_state)

from pprint import pprint

pprint(result)

{'content': 'Obesity has become a major public health concern worldwide, with '
            'rates continuing to rise at an alarming rate. According to the '
            'World Health Organization (WHO), obesity has nearly tripled since '
            '1975, with over 1.9 billion adults being classified as '
            'overweight, and of these, over 650 million are classified as '
            'obese. The impact of obesity extends beyond just individuals, as '
            'it also has significant implications for society as a whole in '
            'terms of healthcare costs, productivity, and overall quality of '
            'life.\n'
            '\n'
            'There are several factors that contribute to the development of '
            'obesity, including genetics, poor diet and nutrition, lack of '
            'physical activity, and environmental factors. While genetics can '
            'play a role in predisposing individuals to obesity, lifestyle '
            'choices such 